## Exploration de séquences contigües à partir de k-mers

Le code suivant a été réalisé dans le cadre du projet 2 du cours de Structure de données.

Il s'agissait de construire un graphe de Brujin permettant de reconstruire des segments possibles d'ADN. Le graphe est construit à partir d'une liste de séquences desquelles sont extraits les k-mers. Par la suite, l'exploration du graphe permet de reconstruire toutes les séquences possibles (sans boucle) débutant à une racine. Une fois ces séquences obtenues, on les compare à des structures de gènes connues qu'on souhaite identifier dans les séquences d'ADN étudiées.

K-Mer : sous-chaîne de k caractères contigües contenue dans une séquence. Pour une séquence de longueur L, il y aura L-k+1 k-mers. Dans le cas de l'ADN, les caractères sont "ATCG" et la séquence "AATGTTGCGAGCGTCGTAGT" contient 16 5-mers, dont "GTTGC" et "GTAGT".

Graphe De Brujin : Graphe où chaque noeud est un k-mer et les successeurs d'un noeud sont les k-mers suivant possibles (par exemple pour le 3-mers "ATA" les successeurs possibles sont "TAA", "TAC", "TAG" et "TAT") présents dans les séquences étudiées et, de même, les parents d'un noeud sont les prédecesseurs du k-mer présents dans les séquences étudiées.

### Particularités du projet

Le graphe de Brujin devait être implanté à l'aide d'un dictionnaire, sans toutefois utiliser les classes built-in de Python dict(), set(), frozenset() etc. Sa structure est contenue dans le fichier graph.py

Le graphe de Brujin construits extrait les 21-mers présents dans le fichier "reads.fastq.gz" contenant plusieurs milliers de séquences de 100 caractères d'ADN. Pour chacune de ces séqences, 80 21-mers sont extraits et rajoutés au graphe, et les pointeurs vers les successeurs/prédecesseurs de chacun de ces 21-mers sont créés au fur et à mesure. Les redondances sont ignorées.

On fois le graphe construit, on identifie tous les noeud sans parent du graphe et on les parcourt pour produire tous les trajets possibles à partir de ces racines, en s'empêchant toutefois de parcourir une boucle. Les semgments contigüs identifiés possibles sont enregistrés dans le fichier "contig.fasta". (possiblement long de plusieurs centaines de caractères)

Finalement, on compare les segments contigüs avec les gènes que l'on désire identifier et on rapporte les occurences. Les gènes en question sont comparés à chacune des séquences pour voir s'ils sont présents dans le segment. Remarquons que les gènes étudiés doivent être plus court (ou de même longueur) que les séquences pouvant les contenir. Les occurences sont rapportés dans le fichier "occurences.bed".

In [1]:
#On importe ici les fichiers dont on aura besoin

from graph import *         #Note: le fichier graph contient un mapping qui doit être loadé pour que la
                                #fonction de hâchage s'effectue correctement
import gzip

all_seq=[]

with gzip.open('reads.fastq.gz',mode='rt') as f:
    for sequid in f:
        seq, _, qual = f.readline(),f.readline(),f.readline()
        seq = seq[:-1]
        all_seq.append(seq)
        
def read_fasta(path):
    with gzip.open(path, 'rt') as f:
        accession, description, seq = None, None, None
        for line in f:
            if line[0] == '>':
            # yield current record
                if accession is not None:
                    yield accession, description, seq
    # start a new record
                accession, description = line[1:].rstrip().split(maxsplit=1)
                seq = ''
            else:
                seq += line.rstrip()

## 2a) Encodage de tous les K-mers

In [2]:
NotreGraph = DeBrujinGraph(all_seq)
print(NotreGraph.load_factor())         #On vérifie que la capacité est respectée initialement

0.499127344768041


# 3 Parcourir le graphe et produire les segments contiguës

In [3]:
# Notre implémentation permet de trouver les racines du graphes (les noeuds sans prédécesseurs)

roots = NotreGraph.find_roots()               #Cette étape prend environ 30 sec. sur ma machine

#En partant des racines, on explore le graphe à partir des racines pour produire les semgents contiguës
#--> Dans l'implémentation de base, aucune boucle à partir d'une même racine n'est tolérée

contig = NotreGraph.walk_all(roots)

print('La plus longue séquence (ou une des) contiguë est:')
print()
print(max(contig,key=lambda x: len(x)))                       #Impression de la plus longue séquence, comme test
                                                                

La plus longue séquence (ou une des) contiguë est:

CAATCGTCCTACGCACCATCATCGTCACGCAACGCCTTCTCTCAATCACCACATCGTGATTGGAGTGCTTCAACTGTCTACGACAGAGTTTATAACTCGAATAGTGATGTAGAGCGCACTCTGTCTCCAAGTGATCCATCGAGAAATCGCAATATTCGAGAGACAACAACCGTCATCCACAGAACTCCGTCGCCAACTGGTTTGAGATCTACTCAGATCACTGAGACAATCACTCGCACAACTCACCGGTCACCGTCACCACCCAGAGGAGTTTCCAACACCTCTTTCGGAAGTGTCAGAACCCAAGAGTTTGCCGAACGTATTGCCCGCTCTCCATCCCCAACTGGATACGAAAGAGAATTTGTTGAGAAGCGAACAACCTATAGGTCACCATCGCCGGCAAGAACATCAAGTGTGGCTCACTCCCATATTAGTGAGGTTCCACTTTCTCCAATTCAAGGAATGGATACTCCGTTTGATCATCATGAACGTGTGAAGAAAGTTGAGAGAATGGATCCATTGGCTGATGAAGAAGAACGTGAACAGAGAAGAAATCAGCTTGAAAGAGATTCTAAAAACACTCTAGGATACACTGTTGCTCAATATGGTGATGGTGAAAAACGATATTTTGGAGACACACTTGAAAAAAAGGATCGACCACCATCTGCTGGATACTACTCATCTGTTCAGCAAAGATCAACATCTCCAGAGTACTCGACAGTTTACGAGCGATATGACAGAAAGAGCGAAAAACCAGATTTGCCGCCAAGAGTTGAAGGTCATGTCTCTGCAAGCTACAAGGGATATGAGCCAGTTTACTCTGAAGTGACAACAACTAGAACCACGACTACTACAGAATACGAGAACATTGACAAAAAACCAAATGTGCCAAAAAAGAGAGCTACTACTCCAGAAGGACACGTTGAGCCAGTAAATGACAATGAGGAA

#### Écriture du fichier fasta

In [4]:
## ------Note ----------
## Le code utilisé pour écrire le fichier fasta a été trouvé et copié (légérement modifié) du site web suivant:
## http://python.omics.wiki/biopython/examples/read-fasta

## On utlise le module uuid de python pour générer des identifiants uniques rapidement

import uuid

file_out='contigs.fasta'
copy_id=[]

with open(file_out, 'w') as f_out:
    for idend,sequence in zip(range(len(contig)),contig):
        # do something (print or edit seq_record)        
        to_print_id = str(uuid.uuid1(idend))
        copy_id.append(to_print_id)
        to_print_seq =  sequence
        # write new fasta file
        f_out.write('>' + to_print_id + ' '+'\n' + to_print_seq+'\n'+'\n')

## 4) Comparaisons des séquences et création du fichier occurences.bed

In [7]:
file_out ='occurences_Final.bed'

to_compared=read_fasta('GCF_000002985.6_WBcel235_rna.fna.gz')
to_compare=[]
for (ref_id,_,ref) in to_compared:
    to_compare.append((ref_id,_,ref))

def compare(seq,i):
    to_print=[]
    for (ref_id,_,ref) in to_compare:
        ind = seq.find(ref)
        if ind >0:
            to_print.append(ref_id+'    '+str(ind+1)+'    '+str(ind+len(ref))+'    '+copy_id[i]+'\n')
    return to_print



with open(file_out, 'w') as f_out:
    for i in range(0,55000,1000):
        for seq,ind in zip(contig[i:i+1000],range(i,i+1000)):
            printing = compare(seq,ind)
            for line in printing:
                f_out.write(line)
        print('Travaillons sur segments contiguës suivants',i,' à ',i+1000)
    print('Dernier strecth!!')
    for seq,ind in zip(contig[55000:],range(55000,55306)):
        printing = compare(seq,ind)
        for line in printing:
            f_out.write(line)
        

Travaillons sur segments contiguës suivants 0  à  1000
Travaillons sur segments contiguës suivants 1000  à  2000
Travaillons sur segments contiguës suivants 2000  à  3000
Travaillons sur segments contiguës suivants 3000  à  4000
Travaillons sur segments contiguës suivants 4000  à  5000
Travaillons sur segments contiguës suivants 5000  à  6000
Travaillons sur segments contiguës suivants 6000  à  7000
Travaillons sur segments contiguës suivants 7000  à  8000
Travaillons sur segments contiguës suivants 8000  à  9000
Travaillons sur segments contiguës suivants 9000  à  10000
Travaillons sur segments contiguës suivants 10000  à  11000
Travaillons sur segments contiguës suivants 11000  à  12000
Travaillons sur segments contiguës suivants 12000  à  13000
Travaillons sur segments contiguës suivants 13000  à  14000
Travaillons sur segments contiguës suivants 14000  à  15000
Travaillons sur segments contiguës suivants 15000  à  16000
Travaillons sur segments contiguës suivants 16000  à  17000
Tr

In [6]:
len(contig)

55306